# Sistema de Modelado y Análisis de Supervivencia (médica)

En este notebook se detalla un sistema o secuencia de pasos a seguir que se pueden aplicar a cualquier modelo de Machine Learning que involucren problemas de tipo ***supervivencia***.

## 1. Dataset
* Cada fila representa a un paciente.
* El objetivo es predecir el riesgo (**risk**) de que un paciente muera
    * Esto se determina en la columna **event**.
---
En los análisis de supervivencia tenemos:
* Los targets:
    * **event**: True/False (muere/no muere).
    * **time**: El tiempo cuando el evento ocurre.
* Explanatory variables:
    * El resto de variables (age, prior_therapy, etc)

Por lo cual la formula de la ecuación matemática del modelo será:
$$
risk = (w_0) + (w_1) \cdot age + (w_2) \cdot prior\_therapy
$$

Con los modelos de Machine Learning, se busca encontrar los mejores valores (optimización) para los pesos ***w1*** y ***w2*** de la ecuación anterior, para finalmente calcular el **risk** asociado.

In [1]:
import pandas as pd

df_patients = pd.read_excel("../data/data_lung_cancer_smote.xlsx")
list_columns_categorical = df_patients.select_dtypes(include="object").columns
df_patients[list_columns_categorical] = df_patients[list_columns_categorical].astype("category")        # Transformación Object a Category. Paso importante para que OneHotEncoder() reconozca las variables categóricas y las transforme.
df_patients

,event,time,age,karnofsky_score,months_from_diagnosis,prior_therapy,treatment,celltype
0,True,2.373626,69.000000,60.000000,7.000000,No,Standard,Squamous
1,True,7.516484,38.000000,60.000000,3.000000,No,Standard,Squamous
2,True,4.153846,63.000000,60.000000,9.000000,Yes,Standard,Squamous
3,True,3.890110,65.000000,70.000000,11.000000,Yes,Standard,Squamous
4,True,0.329670,49.000000,20.000000,5.000000,No,Standard,Squamous
...,...,...,...,...,...,...,...,...
238,False,3.142881,65.810046,64.640822,4.762009,No,Standard,Smallcell
239,False,3.380047,36.495508,70.684273,21.551683,Yes,Test,Smallcell
240,False,3.082424,65.029553,81.087920,4.852974,No,Standard,Squamous
241,False,2.986648,62.424988,77.842548,4.084998,No,Standard,Smallcell


## 2. Feature Selection

Selección de las variables a utilizar en el modelo:
* `y (target)`: **event** y **time**
    * Para que estas variables puedan ser procesadas por el modelo, deben transformarse a otra estructura de datos (*numpy records array*). Esto se hace con *.to_records()*
* `x (explanatory)`: Variables relevantes para calcular el riesgo de un paciente.

### 2.1 Preprocessing Data

1. Revisar **NaN**: Eliminarlos del dataset
2. Transformar los datos categóricos de las variables Exploratory (X) a numéricos con **OneHotEncoder()**
    * La variable target (y) no necesita transformación de categóricos a numéricos ya que cuando se aplica el algoritmo de ML con .fit() este hace la transformación de forma automática.

In [2]:
df_patients.isna().sum()

event                    0
time                     0
age                      0
karnofsky_score          0
months_from_diagnosis    0
prior_therapy            0
treatment                0
celltype                 0
dtype: int64

In [3]:
y = df_patients[["event", "time"]].to_records(index=False)

In [4]:
X = df_patients.drop(["event", "time"], axis=1)
X

,age,karnofsky_score,months_from_diagnosis,prior_therapy,treatment,celltype
0,69.000000,60.000000,7.000000,No,Standard,Squamous
1,38.000000,60.000000,3.000000,No,Standard,Squamous
2,63.000000,60.000000,9.000000,Yes,Standard,Squamous
3,65.000000,70.000000,11.000000,Yes,Standard,Squamous
4,49.000000,20.000000,5.000000,No,Standard,Squamous
...,...,...,...,...,...,...
238,65.810046,64.640822,4.762009,No,Standard,Smallcell
239,36.495508,70.684273,21.551683,Yes,Test,Smallcell
240,65.029553,81.087920,4.852974,No,Standard,Squamous
241,62.424988,77.842548,4.084998,No,Standard,Smallcell


In [5]:
from sksurv.preprocessing import OneHotEncoder

In [6]:
encoder = OneHotEncoder()

In [7]:
X = encoder.fit_transform(X)

### 2.2 Train Test Split

In [8]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [19]:
pd.DataFrame({
    "Dataset": ["X_train", "X_test", "y_train", "y_test"],
    "Registros": [len(X_train), len(X_test), len(y_train), len(y_test)]
})

,Dataset,Registros
0,X_train,170
1,X_test,73
2,y_train,170
3,y_test,73


## 3. The Cox PH Model

### 3.1 Fit - Ajuste del modelo (ecuación matemática)

In [13]:
from sksurv.linear_model import CoxPHSurvivalAnalysis

In [14]:
model_cox = CoxPHSurvivalAnalysis()

In [16]:
model_cox.fit(X, y)     # CORREGIR A FUTURO: Se está utilizando el dataset completo para entrenar, lo cual es un error. Se debe crear Train y Test sets.

CoxPHSurvivalAnalysis()

### 3.2 Predict - Predicciones con el modelo ajustado

In [17]:
model_cox.predict(X)

array([-3.81207507, -3.94068806, -4.03506263, -4.41753498, -2.26935265,
       -3.19785573, -4.44309027, -3.39564103, -4.22251274, -3.80002118,
       -2.81813545, -2.6397711 , -4.8793659 , -4.4698678 , -3.11812674,
       -2.39601994, -4.13522359, -3.1252231 , -2.32615628, -3.08306505,
       -3.24182634, -1.89844748, -3.97474361, -2.13990545, -2.61361835,
       -3.08828229, -4.13042649, -2.55556036, -1.45618697, -3.92762803,
       -1.87356057, -3.71364378, -3.55011321, -3.1413391 , -1.89724821,
       -3.11624761, -3.74580382, -3.07357014, -3.52146804, -2.64586134,
       -2.68656041, -2.34441112, -0.68614969, -2.56888256, -1.38024702,
       -3.10819292, -3.04636519, -2.02008876, -3.00326758, -1.05840313,
       -3.10391525, -2.90279495, -4.00091344, -2.73307977, -3.12932471,
       -2.52460028, -4.05324626, -4.22859882, -3.60826632, -4.43102079,
       -4.38740375, -3.938826  , -4.07479506, -3.81962014, -3.42217593,
       -4.34856912, -4.41039685, -3.36205219, -3.14360201, -3.96

### 3.3 Score - Evaluación de las predicciones

In [18]:
model_cox.score(X, y)

0.7708419620140821

El modelo tiene un rendimiento bastante pobre, solo un 56% de las predicciones son correctas. Con este resultado es incluso casi igual que lanzar una moneda al aire y determinar si el paciente muere o no, ya que las chances del lanzmaiento de la moneda son 50% (true) y 50%(false).

Se crearán nuevos modelos para comparar si alguno de ellos obtiene un mejor Score.  
***No olvidar que el modelo fue entrenado con el 100% de los datos, lo cual es un error. En futuros fix se dividirá en train y test sets.***

## 4. Decision Tree Model

### 4.1 Fit - Ajuste del modelo

In [19]:
from sksurv.tree import SurvivalTree

In [20]:
model_tree = SurvivalTree()

In [21]:
model_tree.fit(X, y)

SurvivalTree()

### 4.2 Predict - Predicciones con el modelo ajustado

In [22]:
model_tree.predict(X)

array([ 51.83333333,  67.43333333,  32.5       ,  29.        ,
       163.66666667,  63.16666667,  51.83333333,  15.33333333,
         0.        , 106.4       , 128.        ,  63.16666667,
         4.        ,  85.33333333,  80.16666667, 163.66666667,
        59.5       ,  36.91666667,  80.96666667,   3.        ,
        32.5       ,  80.96666667,  30.        , 163.66666667,
        51.83333333, 130.        ,  47.66666667,  80.96666667,
       128.        ,  59.5       ,  80.96666667,  99.33333333,
        36.91666667,   3.        ,  80.96666667,  99.33333333,
        26.33333333,  41.83333333,  99.33333333, 106.4       ,
       130.33333333, 166.83333333, 128.        ,  41.83333333,
        63.16666667,  67.43333333,  26.        ,  47.        ,
        17.        , 163.66666667,  67.43333333,  21.83333333,
        21.06666667,  26.33333333,  21.06666667, 166.83333333,
         6.        ,  47.66666667,  45.23333333,   0.        ,
        21.06666667,  45.23333333,  67.43333333,   7.  

### 4.3 Score - Evaluación de las predicciones

In [23]:
model_tree.score(X, y)

0.9161292231228921

## 5. Random Forest Model

### 5.1 Fit - Ajuste del modelo

In [24]:
from sksurv.ensemble import RandomSurvivalForest

In [25]:
model_rf = RandomSurvivalForest()

In [26]:
model_rf.fit(X, y)

RandomSurvivalForest()

### 5.2 Predict - Predicciones con el modelo ajustado

In [27]:
model_rf.predict(X)

array([ 35.53907927,  39.67411472,  38.69757927,  51.06751578,
        95.60828175,  97.39258526,  44.02627005,  33.35814706,
        27.27313086,  55.27884514, 109.72930351,  82.83085907,
        21.69365945,  48.2298254 ,  59.05271254, 103.82806923,
        63.31303345,  54.72728049,  67.88699411,  35.55515149,
        37.36975983, 114.01454503,  47.76815695, 114.34447432,
        53.3984312 ,  68.73639059,  48.6297575 , 107.56023544,
       116.88364286,  74.97920702, 111.56901694,  47.20480552,
        67.80302278,  34.3814692 , 111.76738891,  78.62373505,
        44.39478078,  46.96044563,  73.14617418,  75.08267349,
        38.97002218, 116.03442632, 122.66761022,  47.4428888 ,
       100.80502987,  60.1750159 ,  42.30724612,  67.95124166,
        37.62301183, 112.46923829,  63.70720567,  41.65582979,
        21.83176445,  32.25103491,  32.58203491, 100.40259146,
        29.1012324 ,  39.0911506 ,  29.99516108,  16.56584501,
        17.29509382,  31.13607367,  39.75885145,  26.73

### 5.3 Score - Evaluación de las predicciones

In [28]:
model_rf.score(X, y)

0.9079936098455713

## 6. Resultado de los modelos

In [29]:
columnas = ["Modelo", "Score"]

In [31]:
df_resultados = pd.DataFrame({
    "Modelos": ["Cox PH", "Decision Tree", "Random Forest"],
    "Score": [model_cox.score(X,y), model_tree.score(X,y), model_rf.score(X,y)]})
df_resultados.style.background_gradient()

,Modelos,Score
0,Cox PH,0.770842
1,Decision Tree,0.916129
2,Random Forest,0.907994
